In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications import NASNetLarge
from tensorflow.keras.layers import Dense, InputLayer, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalMaxPooling2D
from tensorflow.keras.preprocessing import image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.model_selection import train_test_split

In [2]:
train = pd.read_csv('train_new.csv')
train = train.sample(n=10000).reset_index(drop=True)
train.head()

,image,class
0,v_PlayingPiano_g15_c04.avi_frame7.jpg,PlayingPiano
1,v_MoppingFloor_g10_c02.avi_frame4.jpg,MoppingFloor
2,v_Kayaking_g17_c05.avi_frame4.jpg,Kayaking
3,v_CricketBowling_g18_c02.avi_frame2.jpg,CricketBowling
4,v_ThrowDiscus_g18_c02.avi_frame1.jpg,ThrowDiscus


In [3]:
# creating an empty list
train_image = []

# for loop to read and store frames
for i in tqdm(range(train.shape[0])):
    # loading the image and keeping the target size as (224,224,3)
    img = image.load_img('train_1/'+train['image'][i], target_size=(224,224,3))
    # converting it to array
    img = image.img_to_array(img)
    # normalizing the pixel value
    img = img/255
    # appending the image to the train_image list
    train_image.append(img)
    
# converting the list to numpy array
X = np.array(train_image)

# shape of the array
X.shape

100%|███████████████████████████████████████████████████████████████████████████| 10000/10000 [00:45<00:00, 220.07it/s]


(10000, 224, 224, 3)

In [4]:
# separating the target
y = train['class']

# creating the training and validation set
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2, stratify = y)

In [5]:
# creating dummies of target variable for train and validation set
y_train = pd.get_dummies(y_train)
y_test = pd.get_dummies(y_test)

In [6]:
# creating the base model of pre-trained VGG16 model
base_model = NASNetLarge(weights='imagenet', include_top=False)

343613440/343610240 [==============================] - 9s 0us/step


In [7]:
# extracting features for training frames
X_train = base_model.predict(X_train)
X_train.shape

(8000, 7, 7, 4032)

In [8]:
# extracting features for validation frames
X_test = base_model.predict(X_test)
X_test.shape

(2000, 7, 7, 4032)

In [10]:
# reshaping the training as well as validation frames in single dimension
X_train = X_train.reshape(8000, 7*7*4032)
X_test = X_test.reshape(2000, 7*7*4032)

In [11]:
# normalizing the pixel values
max = X_train.max()
X_train = X_train/max
X_test = X_test/max

In [12]:
# shape of images
X_train.shape

(8000, 197568)

In [17]:
#defining the model architecture
model = Sequential()
model.add(Dense(1024, activation='relu', input_shape=(197568,)))
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(101, activation='softmax'))

In [18]:
# defining a function to save the weights of best model
from tensorflow.keras.callbacks import ModelCheckpoint
mcp_save = ModelCheckpoint('NASNetLargeweight.hdf5', save_best_only=True, monitor='val_loss', mode='min')

In [19]:
# compiling the model
model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])

In [20]:
# training the model
model.fit(X_train, y_train, epochs=200, validation_data=(X_test, y_test), callbacks=[mcp_save], batch_size=128)

Epoch 1/200
63/63 [==============================] - 50s 790ms/step - loss: 4.3677 - accuracy: 0.0642 - val_loss: 3.8965 - val_accuracy: 0.2390
Epoch 2/200
63/63 [==============================] - 55s 876ms/step - loss: 3.6849 - accuracy: 0.1885 - val_loss: 2.9714 - val_accuracy: 0.3835
Epoch 3/200
63/63 [==============================] - 54s 859ms/step - loss: 2.9876 - accuracy: 0.3136 - val_loss: 2.2185 - val_accuracy: 0.5325
Epoch 4/200
63/63 [==============================] - 54s 864ms/step - loss: 2.4096 - accuracy: 0.4201 - val_loss: 1.7470 - val_accuracy: 0.6015
Epoch 5/200
63/63 [==============================] - 54s 857ms/step - loss: 1.9671 - accuracy: 0.5056 - val_loss: 1.3862 - val_accuracy: 0.6790
Epoch 6/200
63/63 [==============================] - 54s 859ms/step - loss: 1.6326 - accuracy: 0.5698 - val_loss: 1.1555 - val_accuracy: 0.7330
Epoch 7/200
63/63 [==============================] - 54s 858ms/step - loss: 1.3857 - accuracy: 0.6212 - val_loss: 0.9824 - val_accuracy:

63/63 [==============================] - 49s 775ms/step - loss: 0.1271 - accuracy: 0.9680 - val_loss: 0.5412 - val_accuracy: 0.9035
Epoch 58/200
63/63 [==============================] - 49s 784ms/step - loss: 0.1188 - accuracy: 0.9675 - val_loss: 0.5232 - val_accuracy: 0.9070
Epoch 59/200
63/63 [==============================] - 50s 792ms/step - loss: 0.1243 - accuracy: 0.9695 - val_loss: 0.5067 - val_accuracy: 0.9105
Epoch 60/200
63/63 [==============================] - 50s 788ms/step - loss: 0.1198 - accuracy: 0.9697 - val_loss: 0.4926 - val_accuracy: 0.9090
Epoch 61/200
63/63 [==============================] - 50s 796ms/step - loss: 0.1297 - accuracy: 0.9674 - val_loss: 0.4890 - val_accuracy: 0.9100
Epoch 62/200
63/63 [==============================] - 50s 799ms/step - loss: 0.1365 - accuracy: 0.9685 - val_loss: 0.4876 - val_accuracy: 0.9140
Epoch 63/200
63/63 [==============================] - 50s 797ms/step - loss: 0.1182 - accuracy: 0.9697 - val_loss: 0.4983 - val_accuracy: 0.913

63/63 [==============================] - 51s 813ms/step - loss: 0.1078 - accuracy: 0.9758 - val_loss: 0.6253 - val_accuracy: 0.9090
Epoch 114/200
63/63 [==============================] - 52s 819ms/step - loss: 0.1036 - accuracy: 0.9774 - val_loss: 0.6141 - val_accuracy: 0.9135
Epoch 115/200
63/63 [==============================] - 51s 813ms/step - loss: 0.0983 - accuracy: 0.9784 - val_loss: 0.5925 - val_accuracy: 0.9135
Epoch 116/200
63/63 [==============================] - 51s 815ms/step - loss: 0.0995 - accuracy: 0.9769 - val_loss: 0.5908 - val_accuracy: 0.9110
Epoch 117/200
63/63 [==============================] - 51s 811ms/step - loss: 0.1084 - accuracy: 0.9764 - val_loss: 0.5872 - val_accuracy: 0.9105
Epoch 118/200
63/63 [==============================] - 51s 808ms/step - loss: 0.1114 - accuracy: 0.9781 - val_loss: 0.5789 - val_accuracy: 0.9110
Epoch 119/200
63/63 [==============================] - 51s 810ms/step - loss: 0.1063 - accuracy: 0.9775 - val_loss: 0.5770 - val_accuracy:

63/63 [==============================] - 51s 804ms/step - loss: 0.0902 - accuracy: 0.9795 - val_loss: 0.6819 - val_accuracy: 0.9160
Epoch 170/200
63/63 [==============================] - 51s 807ms/step - loss: 0.1043 - accuracy: 0.9808 - val_loss: 0.7407 - val_accuracy: 0.9115
Epoch 171/200
63/63 [==============================] - 52s 822ms/step - loss: 0.1028 - accuracy: 0.9810 - val_loss: 0.6567 - val_accuracy: 0.9150
Epoch 172/200
63/63 [==============================] - 53s 839ms/step - loss: 0.0991 - accuracy: 0.9795 - val_loss: 0.7094 - val_accuracy: 0.9160
Epoch 173/200
63/63 [==============================] - 51s 816ms/step - loss: 0.1168 - accuracy: 0.9796 - val_loss: 0.7292 - val_accuracy: 0.9125
Epoch 174/200
63/63 [==============================] - 45s 715ms/step - loss: 0.0862 - accuracy: 0.9795 - val_loss: 0.7583 - val_accuracy: 0.9150
Epoch 175/200
63/63 [==============================] - 44s 698ms/step - loss: 0.0845 - accuracy: 0.9837 - val_loss: 0.7146 - val_accuracy:

In [21]:
# loading the trained weights
model.load_weights("NASNetLargeweight.hdf5")

In [22]:
# compiling the model
model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])

In [24]:
# getting the test list
f = open("ucfTrainTestlist/testlist01.txt", "r")
temp = f.read()
videos = temp.split('\n')

# creating the dataframe
test = pd.DataFrame()
test['video_name'] = videos
test = test[:-1]
test_videos = test['video_name']
test.head()

,video_name
0,ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c01.avi
1,ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c02.avi
2,ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c03.avi
3,ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c04.avi
4,ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c05.avi


In [25]:
# creating the tags
train = pd.read_csv('train_new.csv')
y = train['class']
y = pd.get_dummies(y)

In [26]:
import cv2
import math
import os
from glob import glob
from scipy import stats as s

In [28]:
# creating two lists to store predicted and actual tags
predict = []
actual = []

# for loop to extract frames from each test video
for i in tqdm(range(test_videos.shape[0])):
    count = 0
    videoFile = test_videos[i]
    cap = cv2.VideoCapture('UCF-101/'+videoFile.split(' ')[0])   # capturing the video from the given path
    frameRate = cap.get(5) #frame rate
    x=1
    # removing all other files from the temp folder
    files = glob('temp/*')
    for f in files:
        os.remove(f)
    while(cap.isOpened()):
        frameId = cap.get(1) #current frame number
        ret, frame = cap.read()
        if (ret != True):
            break
        if (frameId % math.floor(frameRate) == 0):
            # storing the frames of this particular video in temp folder
            filename ='temp/' + "_frame%d.jpg" % count;count+=1
            cv2.imwrite(filename, frame)
    cap.release()
    
    # reading all the frames from temp folder
    images = glob("temp/*.jpg")
    
    prediction_images = []
    for i in range(len(images)):
        img = image.load_img(images[i], target_size=(224,224,3))
        img = image.img_to_array(img)
        img = img/255
        prediction_images.append(img)
        
    # converting all the frames for a test video into numpy array
    prediction_images = np.array(prediction_images)
    # extracting features using pre-trained model
    prediction_images = base_model.predict(prediction_images)
    # converting features in one dimensional array
    prediction_images = prediction_images.reshape(prediction_images.shape[0], 7*7*4032)
    # predicting tags for each array
    prediction = model.predict_classes(prediction_images)
    # appending the mode of predictions in predict list to assign the tag to the video
    predict.append(y.columns.values[s.mode(prediction)[0][0]])
    # appending the actual tag of the video
    actual.append(videoFile.split('/')[1].split('_')[1])

  0%|                                                                                         | 0/3783 [00:00<?, ?it/s]

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


100%|████████████████████████████████████████████████████████████████████████████| 3783/3783 [1:02:20<00:00,  1.01it/s]


In [29]:
# checking the accuracy of the predicted tags
from sklearn.metrics import accuracy_score
accuracy_score(predict, actual)*100

69.15146708961142